# Observatory Data

<a id="top"/>

## Contents

- [Settings and functions](#settings)
- [Hourly mean values](#obs)
    - [Read data from ASCII files](#obs-read-ascii)
    - [Read data from multiple files](#obs-multifiles)
- [Minute and second mean values](#obsms)
    - [Read data from CDF files](#obsms-read-cdf)
    - [Read data from multiple files](#obsms-multifiles)

<a id="settings" />

## Settings and functions

[[TOP]](#top)

In [ ]:
# Python standard library
import re
from contextlib import closing
from datetime import datetime
from pathlib import Path

# Extra libraries
import cdflib
import numpy as np
import pandas as pd


# TODO: update the data dir once the files will be available in the shared folder
OBS_HOUR_DIR = Path('~/data/AUX_OBS/hour').expanduser()
OBS_MINUTE_DIR = Path('~/data/AUX_OBS/minute').expanduser()
OBS_SECOND_DIR = Path('~/data/AUX_OBS/second').expanduser()


def ascii_to_pandas(file):
    """Convert an OBS ASCII file to a pandas DataFrame.
    
    Parameters
    ----------
    file : str or os.PathLike
        OBS ASCII file.
    
    Returns
    -------
    pandas.DataFrame
        data contained in the OBS ASCII file.

    """
    df = pd.read_csv(
        file,
        comment='#',
        delim_whitespace=True,
        names = ['IAGA_code', 'Latitude', 'Longitude', 'Radius',
                 'yyyy', 'mm', 'dd', 'UT', 'B_N', 'B_E', 'B_C'],
        parse_dates={'Timestamp': [4, 5, 6]},
        infer_datetime_format=True
    )
    df['Timestamp'] = df['Timestamp'] + pd.to_timedelta(df['UT'], 'h')
    df.drop(columns='UT', inplace=True)
    df.set_index('Timestamp', inplace=True)
    return df


def cdf_to_pandas(file):
    """Convert an OBS CDF file to a pandas DataFrame.
    
    Parameters
    ----------
    file : str or os.PathLike
        OBS CDF file.
    
    Returns
    -------
    pandas.DataFrame
        data contained in the OBS CDF file.

    """
    with closing(cdflib.cdfread.CDF(file)) as data:
        ts = pd.DatetimeIndex(
            cdflib.cdfepoch.encode(data.varget('Timestamp'), iso_8601=True),
            name='Timestamp'
        )
        df = pd.DataFrame(
            {
                'IAGA_code': data.varget('IAGA_code')[:,0,0],
                'Latitude': data.varget('Latitude'),
                'Longitude': data.varget('Longitude'),
                'Radius': data.varget('Radius'),
                'B_N': data.varget('B_NEC')[:,0],
                'B_E': data.varget('B_NEC')[:,1],
                'B_C': data.varget('B_NEC')[:,2]
            },
            index=ts
        )
    return df

<a id="obs" />

## Hourly mean values

[[TOP]](#top)

Repository:
- ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/hour/

<a id="obs-read-ascii" />

### Read data from ASCII files

[[TOP]](#top)

In [ ]:
# NBVAL_SKIP
# OPTIONAL - download data from the FTP server
!wget -nv -nc -P ~/data/AUX_OBS/hour ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/hour/SW_OPER_AUX_OBS_2__201[89]*
!find ~/data/AUX_OBS -name "*.ZIP" | while read f ; do unzip -u $f -d `dirname $f` ; done
!find ~/data/AUX_OBS -name "*.ZIP" -delete
!find ~/data/AUX_OBS -name "*.HDR" -delete

Select one of the AUX_OBS_2_ files (e.g. the first one):

In [ ]:
file1 = sorted(OBS_HOUR_DIR.glob('SW_OPER_AUX_OBS_2_*'))[0]

file1

Read ASCII file and convert data to a `pandas.DataFrame`:

In [ ]:
df1 = pd.read_csv(
    file1,
    comment='#',
    delim_whitespace=True,
    names = ['IAGA_code', 'Latitude', 'Longitude',
             'Radius', 'yyyy', 'mm', 'dd', 'UT', 'B_N', 'B_E', 'B_C'],
    parse_dates={'Timestamp': [4, 5, 6]},
    infer_datetime_format=True
)
df1['Timestamp'] = df1['Timestamp'] + pd.to_timedelta(df1['UT'], 'h')
df1.drop(columns='UT', inplace=True)
df1.set_index('Timestamp', inplace=True)

df1

For more information on `pandas.Dataframe` see: https://pandas.pydata.org/docs/reference/frame.

The same result can be obtained with the `ascii_to_pandas()` function defined above (see [Settings and functions](#settings)).

In [ ]:
new = ascii_to_pandas(file1)

new

Compare the two data frames:

In [ ]:
pd.testing.assert_frame_equal(df1, new)

Example: get minimum and maximum dates:

In [ ]:
df1.index.min(), df1.index.max()

Example: get list of observatories (IAGA codes) stored in the files:

In [ ]:
df1['IAGA_code'].unique()

<a id="obs-multifiles" />

### Read data from multiple files

[[TOP]](#top)

Pandas dataframes can be concatenated to represent data obtained from more than one file. E.g. read data from the next AUX_OBS_2_ file:

In [ ]:
file2 = sorted(OBS_HOUR_DIR.glob('SW_OPER_AUX_OBS_2_*.txt'))[1]

df2 = ascii_to_pandas(file2)

df2

The two dataframes can be concatenated using the `pandas.concat()` function (for more information see: https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas.concat):

In [ ]:
concatenated = pd.concat([df1, df2])
concatenated.sort_values(by=['IAGA_code', 'Timestamp'], inplace=True)

concatenated.index.min(), concatenated.index.max()

In [ ]:
concatenated

<a id="obsms" />

## Minute and second mean values

[[TOP]](#top)

Files containing observatory minute and second mean values have CDF format. They can be downloade from:

- ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/minute/
- ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/second/

<a id="obsms-read-cdf" />

### Read data from CDF files

[[TOP]](#top)

In [ ]:
# NBVAL_SKIP
# OPTIONAL - download data from the FTP server
!wget -nv -nc -P ~/data/AUX_OBS/minute ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/minute/SW_OPER_AUX_OBSM2__201912*
!wget -nv -nc -P ~/data/AUX_OBS/second ftp://ftp.nerc-murchison.ac.uk/geomag/Swarm/AUX_OBS/second/SW_OPER_AUX_OBSS2__201912*
!find ~/data/AUX_OBS -name "*.ZIP" | while read f ; do unzip -u $f -d `dirname $f` ; done
!find ~/data/AUX_OBS -name "*.ZIP" -delete
!find ~/data/AUX_OBS -name "*.HDR" -delete

Select one of the AUX_OBSM2_ files (e.g. the first one):

In [ ]:
file1 = sorted(OBS_MINUTE_DIR.glob('SW_OPER_AUX_OBSM2_*.DBL'))[0]

Read CDF file using `cdflib` (for more information on `cdflib`, see: https://github.com/MAVENSDC/cdflib)

In [ ]:
data = cdflib.CDF(file1)

Get info about the file as a Python dictionary:

In [ ]:
data.cdf_info()

You can see that measurements are stored as *zVariables*:

In [ ]:
data.cdf_info()['zVariables']

Data can be retrieved via the `.varget()` method, e.g:

In [ ]:
data.varget('B_NEC')

Data is returned as a `numpy.ndarray` object (for more information on `numpy.ndarray`, see: https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html).

Variable attributes can be retrieved using the `.varattsget()` method, e.g.:

In [ ]:
data.varattsget('B_NEC')

Attributes are returned as a Python dictionary.

Let's retrieve the timestamps:

In [ ]:
data.varget('Timestamp')

`Timestamp` type is:

In [ ]:
data.varget('Timestamp').dtype

Timestamps are represented as NumPy `float64` values. Why? Get info about `Timestamp` variable using the `.varinq()` method:

In [ ]:
data.varinq('Timestamp')

The returned dictionary shows that the data type is *CDF_EPOCH* consising in a floating point value representing the number of milliseconds since 01-Jan-0000 00:00:00.000. It can be converted to a more readable format (list of strings) using the `cdflib.cdfepoch.encode()` function:

In [ ]:
ts = cdflib.cdfepoch.encode(data.varget('Timestamp'), iso_8601=True)

ts[:5]

Or to a numpy array of `numpy.datetime64` values:

In [ ]:
ts = np.array(cdflib.cdfepoch.encode(data.varget('Timestamp'), iso_8601=True), dtype='datetime64')

ts[:5]

You may be interested also in the CDF global attributes:

In [ ]:
data.globalattsget()

Close the file when you have finished:

In [ ]:
data.close()

AUX_OBSS2_ data contains the same variables:

In [ ]:
with closing(cdflib.cdfread.CDF(list(OBS_SECOND_DIR.glob('SW_OPER_AUX_OBSS2_*.DBL'))[0])) as data:
    zvariables = data.cdf_info()['zVariables']

zvariables

Data can be represented as a `pandas.DataFrame` object:

In [ ]:
with closing(cdflib.cdfread.CDF(file1)) as data:
    ts = pd.DatetimeIndex(
            cdflib.cdfepoch.encode(data.varget('Timestamp'), iso_8601=True),
            name='Timestamp'
        )
    df1 = pd.DataFrame(
        {
            'IAGA_code': data.varget('IAGA_code')[:,0,0],
            'Latitude': data.varget('Latitude'),
            'Longitude': data.varget('Longitude'),
            'Radius': data.varget('Radius'),
            'B_N': data.varget('B_NEC')[:,0],
            'B_E': data.varget('B_NEC')[:,1],
            'B_C': data.varget('B_NEC')[:,2]
        },
        index=ts
    )

df1

For more information on `pandas.Dataframe` see: https://pandas.pydata.org/docs/reference/frame.

The same result can be obtained with the `cdf_to_pandas()` function defined above (see [Settings and functions](#settings)).

In [ ]:
new = cdf_to_pandas(file1)

new

Compare the two data frames:

In [ ]:
pd.testing.assert_frame_equal(df1, new)

Example: get minimum and maximum dates:

In [ ]:
df1.index.min(), df1.index.max()

Example: get list of observatories (IAGA codes) stored in the files:

In [ ]:
df1['IAGA_code'].unique()

Example: get list of observatories (IAGA codes) included in the following ranges of coordinates:
- $30 \leq Latitude \leq 70$
- $-10 \leq Longitude \leq 40$

In [ ]:
df1[(df1['Latitude'] >= 30) & (df1['Latitude'] <= 70) & (df1['Longitude'] >= -10) & (df1['Longitude'] <= 40)]['IAGA_code'].unique()

You can do the same using the `.query()` method (see: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html#pandas.DataFrame.query):

In [ ]:
df1.query('(30 <= Latitude <= 70) and (-10 <= Longitude <= 40)')['IAGA_code'].unique()

<a id="obsms-multifiles" />

### Read data from multiple files

[[TOP]](#top)

Pandas dataframes can be concatenated to represent data obtained from more than one file. E.g. read data from the next AUX_OBSM2_ file:

In [ ]:
file2 = sorted(OBS_MINUTE_DIR.glob('SW_OPER_AUX_OBSM2_*.DBL'))[1]

df2 = cdf_to_pandas(file2)

df2

The two dataframes can be concatenated using the `pandas.concat()` function (for more information see: https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas.concat):

In [ ]:
concatenated = pd.concat([df1, df2])
concatenated.sort_values(by=['IAGA_code', 'Timestamp'], inplace=True)

concatenated.index.min(), concatenated.index.max()

In [ ]:
concatenated

With AUX_OBSS2_ data:

In [ ]:
files = sorted(OBS_SECOND_DIR.glob('SW_OPER_AUX_OBSS2_*.DBL'))[:2]

files

In [ ]:
concatenated = pd.concat([cdf_to_pandas(file) for file in files])
concatenated.sort_values(by=['IAGA_code', 'Timestamp'], inplace=True)

concatenated.index.min(), concatenated.index.max()

In [ ]:
concatenated